In [2]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import random
import math
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten,BatchNormalization,Dropout


ModuleNotFoundError: No module named 'tensorflow'

In [2]:
plt.set_cmap('gray')

<Figure size 640x480 with 0 Axes>

# Loading the dataset: 

In [3]:
def display(images,a):
        m,n = 1, int(len(images)/a)
        for i in range(0,len(images),m*n):
            fig = plt.figure(figsize=(15,8))
            for j in range(m*n):
                fig.add_subplot(m,n,j+1)
                plt.imshow(images[i+j])
                plt.axis('off')
            plt.show()

In [4]:
def split_data(data,y,n):
  images = []
  Y = np.array([4])
  for i in np.unique(y):
    cl = np.where(y == i)
    indx = np.random.choice(cl[0],size = n,replace = False)
    images.append(data[indx,:])
    Y = np.append(Y,np.full(n,i))
  tr = []
  for l in images:
    for item in l:
      tr.append(item)
  tr = np.array(tr)
  return tr,Y[1:]


In [5]:
import glob
data = glob.glob('notebook/dataset/*/*.jpg')

# Data Cleaning:

## We only use images that clearly display the face and both eyes: 

In [6]:
# Load the pre-trained Haar cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

In [7]:
# the function that detect the face in the image:
def detect_face(image_path):
    img = cv2.imread(image_path)
    if img is not None:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Detect faces in the image
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        for (x,y,w,h) in faces:
            roi_gray = gray[y:y+h,x:x+w]
            roi_color = img[y:y+h,x:x+w]
            eyes = eye_cascade.detectMultiScale(roi_gray)
            if len(eyes) ==2:
                return roi_color

In [8]:
import os
data_path = '/kaggle/input/celeberties-dataset'
img_dirs = []
for entry in os.scandir(data_path):
    if entry.is_dir():
        img_dirs.append(entry.path)

img_dirs

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/celeberties-dataset'

In [ ]:
import shutil
if os.path.exists(data_path):
     shutil.rmtree(path_to_cr_data)
os.mkdir(path_to_cr_data)
cropped_image_dirs = []
celebrity_file_names_dict = {}
for img_dir in img_dirs:
    count = 1
    celebrity_name = img_dir.split('/')[-1]
    celebrity_file_names_dict[celebrity_name] = []
    for entry in os.scandir(img_dir):
        roi_color = get_cropped_image_if_2_eyes(entry.path)
        if roi_color is not None:
            cropped_folder = path_to_cr_data + celebrity_name
            if not os.path.exists(cropped_folder):
                os.makedirs(cropped_folder)
                cropped_image_dirs.append(cropped_folder)
                print("Generating cropped images in folder: ",cropped_folder)
            cropped_file_name = celebrity_name + str(count) + ".png"
            cropped_file_path = cropped_folder + "/" + cropped_file_name
            cv2.imwrite(cropped_file_path, roi_color)
            celebrity_file_names_dict[celebrity_name].append(cropped_file_path)
            count += 1

In [ ]:
images = [plt.imread(img) for img in dataset]

# Display a subset of the images
display(images[2:10], 1)

# Labeled the images:

In [ ]:
# Create a mapping for labels
label_map = {'Mahira Khan': 0, 'Han Ye-seul': 1, 'Riyad Mahrez': 2, 'Elyas MBarek': 3, 'Ramy Youssef': 4}

# Assign labels using a list comprehension
Y = np.array([label_map[x.split('/')[-2]] for x in data])

# Convert images to numpy array (if not already)
images = np.array(images)

# Shuffle the dataset
images, Y = shuffle(images, Y, random_state=0)


# Split the images into train and test:

In [ ]:
train,test ,ytrain,ytest =train_test_split(images, Y, train_size=0.8)

# CNN model:

In [ ]:
CNN = Sequential()

CNN.add(Conv2D(32,(3,3),1,activation = 'relu',input_shape=(128,128,3)))
CNN.add(Conv2D(64,(3,3),1,activation = 'relu'))#,padding = 'same',kernel_regularizer = regularizers.l2(0.01)))
CNN.add(MaxPooling2D())
CNN.add(BatchNormalization())

CNN.add(Conv2D(128,(3,3),1,activation = 'relu'))#,padding = 'same',kernel_regularizer = regularizers.l2(0.01)))
CNN.add(MaxPooling2D())
CNN.add(Dropout(0.6))
CNN.add(Conv2D(128,(3,3),1,activation = 'relu'))#,padding = 'same',kernel_regularizer = regularizers.l2(0.01)))
CNN.add(MaxPooling2D())
CNN.add(BatchNormalization())

CNN.add(Flatten())
CNN.add(Dropout(0.2))
CNN.add(Dense(256,activation='relu'))#,kernel_regularizer = regularizers.l2(0.01)))
CNN.add(BatchNormalization())
CNN.add(Dense(5,activation = 'softmax'))

In [ ]:
opt = tf.keras.optimizers.Adam(clipvalue = 0.5)
CNN.compile(optimizer =opt2,loss = tf.losses.SparseCategoricalCrossentropy(),metrics = ['accuracy'])

In [ ]:
CNN.summary()

# Train the model:

In [ ]:
hist = CNN.fit(img_train,Target.reshape(-1,1),epochs = 50, validation_split=0.15)

# Plot the results:

In [ ]:
plt.plot(hist.history['loss'], label= 'train loss')
plt.plot(hist.history['val_loss'], label= 'val loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(hist.history['accuracy'], label= 'train accuracy')
plt.plot(hist.history['val_accuracy'], label= 'val accuracy')
plt.legend()
plt.show()

In [ ]:
def predictions(y):
    pred = []
    for i in range(0,len(y),100):
        x = y[i:i+100]
        p = np.argmax(x)
        pred.append(p)
    pred = np.array(pred)    
    return pred    

def accuracy(y,pr):
    a = sum(pr==y)/ y.size
    return a

In [ ]:
ypred = CNN.predict(test).reshape(-1)
pred = predictions(ypred)
accuracy(Yt,pred)